In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
# リスト 3.3.x 日本語用インデックスの登録（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・kuromoji + ICU を用いた日本語向けアナライザを Elasticsearch 側に定義し、
#     検索時用（jpn-search）と索引用（jpn-index）の2系統で設定する最小例。
#
# 前提/注意:
#   ・このコードは Python クライアント（elasticsearch-py）から ES に送る設定 JSON を組み立てて実行する。
#   ・kuromoji/icu が使用可能な ES（プラグイン内蔵の公式ディストリ or Elastic Cloud 等）を想定。
#   ・"user_dictionary": "my_jisho.dic" は **ESノード側のファイルパス**。ローカルPC上のパスではない点に注意。
#     管理サービスではユーザー辞書ファイルの配置に制約がある（不可/代替手段が必要）ことが多い。
#   ・同義語（synonyms_filter）は初期は空配列 → 実運用では再インデックス or エイリアス切替で更新する設計が安全。
#   ・本スニペットは settings（analysis）のみ定義。**mappings で各 text フィールドへ
#     analyzer/search_analyzer を割り当てない限り効果は出ない**（下部コメント参照）。
# -----------------------------------------------------------------------------------

# インデックス作成用JSONの定義
create_index = {
    "settings": {
        "analysis": {
            # ---------------------------
            # Token Filter 定義セクション
            # ---------------------------
            "filter": {
                "synonyms_filter": {  # 同義語フィルタ
                    "type": "synonym",
                    # "synonyms": [...] に「A, B」「C => D」等で定義。
                    # 空配列の現状では no-op（効果なし）。運用開始後の更新は再作成/エイリアス切替が基本。
                    "synonyms": [
                        # 例: "寿司, すし", "蕎麦, そば", "コンピューター, コンピュータ"
                    ],
                }
                # 例: 固有名詞保護が必要なら keyword_marker を追加し、stemmer より前段に置く
                # "ja_protected": {
                #     "type": "keyword_marker",
                #     "protected_words": ["メーラー","プレイヤー"]  # 長音削減の対象から除外したい語
                # }
            },
            # ----------------------------
            # Tokenizer 定義セクション
            # ----------------------------
            "tokenizer": {
                "kuromoji_w_dic": {  # ユーザー辞書付き kuromoji トークナイザ
                    "type": "kuromoji_tokenizer",  # ← ベースは kuromoji_tokenizer（綴り注意）
                    # ユーザー辞書（*.dic = ビルド済み辞書）を追加。パスは ES ノード側の参照範囲。
                    "user_dictionary": "my_jisho.dic",
                }
            },
            # --------------------------
            # Analyzer 定義セクション
            # --------------------------
            "analyzer": {
                # 検索用（想起重視: synonyms を含める）
                "jpn-search": {
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer",  # 文字単位の正規化（NFKC等：全角/半角・互換文字の揺れ吸収）
                        "kuromoji_iteration_mark",  # 繰り返し記号（々/ゝ/ヽ 等）の展開
                    ],
                    "tokenizer": "kuromoji_w_dic",  # ユーザー辞書込みの形態素解析
                    "filter": [
                        "synonyms_filter",  # 同義語展開（想起向上）。定義が空なら実質無効
                        "kuromoji_baseform",  # 活用語の原形化（食べた→食べる）
                        "kuromoji_part_of_speech",  # 不要品詞の除去（助詞/助動詞など）※ stoptags 既定に依存
                        "ja_stop",  # 日本語ストップワードの除去
                        "kuromoji_number",  # 数表現の正規化（漢数字→算用数字等）
                        "kuromoji_stemmer",  # カタカナ語の長音正規化（コンピューター→コンピュータ）
                    ],
                },
                # 索引用（精度重視: 通常は synonyms を含めない）
                "jpn-index": {
                    "type": "custom",
                    "char_filter": ["icu_normalizer", "kuromoji_iteration_mark"],
                    "tokenizer": "kuromoji_w_dic",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_part_of_speech",
                        "ja_stop",
                        "kuromoji_number",
                        "kuromoji_stemmer",
                        # ※ 必要に応じてここでは synonyms を外すことで、インデックス語彙を素直に保ち、
                        #    検索時だけ同義語で想起を伸ばす設計が一般的（観測性/再構成容易性の観点）。
                    ],
                },
            },
        }
    }
}

# 日本語用インデックス名の定義
jp_index = "jp_index"

# 既存インデックスの削除（検証用途）
# 本番運用では危険操作。誤削除防止のため、対象インデックスの限定やスナップショット取得を徹底すること。
if es.indices.exists(index=jp_index):
    es.indices.delete(index=jp_index)

# インデックス jp_index の生成（settings のみ）
# 実運用では mappings でフィールドへ analyzer/search_analyzer を割り当てること。
# 例）マッピングの一例（参考: 実際には create に body で併せて渡す）:
# {
#   "mappings": {
#     "properties": {
#       "title":   { "type": "text", "analyzer": "jpn-index", "search_analyzer": "jpn-search" },
#       "content": { "type": "text", "analyzer": "jpn-index", "search_analyzer": "jpn-search" },
#       "brand":   { "type": "keyword" }  # 正確一致が欲しいフィールドは keyword を併用
#     }
#   }
# }
es.indices.create(index=jp_index, body=create_index)

# （動作確認のヒント）
# - _analyze API でトークンを確認:
#   es.indices.analyze(index=jp_index, body={"analyzer":"jpn-index","text":"コンピューターを操作する"})
# - synonyms を更新したい場合は、新インデックスを作って reindex → エイリアス切替が安全（ゼロダウンタイム）。

In [ ]:
# 分析結果表示用関数（Elasticsearch _analyze を用いた日本語トークン列の取得）
# -----------------------------------------------------------------------------------
# 目的:
#   ・インデックス jp_index に定義済みの検索用アナライザ "jpn-search" を使って、入力文字列を
#     形態素解析＋各種フィルタ（原形化・品詞/ストップ語除去・数値/長音正規化 等）に通し、
#     生成されたトークンの「語彙（token 文字列）」だけを配列として返す。
#
# 理論メモ:
#   ・_analyze は「char_filter → tokenizer → filter」の順に適用される検索前処理のシミュレーション。
#   ・ここでは検索時アナライザ（jpn-search）を明示するため、**index と analyzer を必ず指定**している。
#     これによりインデックス固有のユーザー辞書・同義語・stoptags などが反映される。
#   ・戻り値 ret["tokens"] は各トークンの詳細辞書（token, position, start_offset, end_offset, type 等）。
#     学習用に token 文字列のみを抽出して返すが、デバッグ時は position/offset を見ると挙動が把握しやすい。
#
# 前提:
#   ・es: Elasticsearch クライアント（例: es = Elasticsearch(...)）
#   ・jp_index: インデックス名（例: 'jp_index'）
#   ・"jpn-search": jp_index の settings.analysis.analyzer に定義済みであること
# -----------------------------------------------------------------------------------


def analyse_jp_text(text):
    # _analyze API に渡すリクエストボディを構築
    # - analyzer: 検索用アナライザ（"jpn-search"）。同義語展開や原形化など検索時の処理を再現
    # - text    : 解析対象テキスト
    body = {"analyzer": "jpn-search", "text": text}

    # 指定インデックスのアナライザで解析を実行
    # 例外例:
    #  - RequestError(400): 指定アナライザが存在しない / ユーザー辞書の読込失敗 など
    #  - ConnectionError: ES ノード未起動・接続情報不備
    ret = es.indices.analyze(index=jp_index, body=body)

    # 返却ペイロードから tokens 配列を取得
    tokens = ret["tokens"]

    # 各トークン辞書の "token" フィールド（語彙）だけを抽出して返す
    tokens2 = [token["token"] for token in tokens]
    return tokens2


# 関数のテスト
# 期待例（設定依存）: 「関数」「テスト」などの内容語が残り、助詞/記号は除去されやすい
print(analyse_jp_text("関数のテスト"))

# （デバッグ補助: 位置情報も見たい場合の例。必要時にコメントアウトを外す）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"関数のテスト"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))

In [ ]:
# リスト 3.3.12 マッピングの設定（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・日本語用インデックス（jp_index）に対し、テキストフィールド "content" のマッピングを設定する。
#   ・索引用アナライザ（"jpn-index"）と検索用アナライザ（"jpn-search"）をフィールドに紐付ける。
#
# 理論メモ（重要ポイント）:
#   1) text と keyword の住み分け
#      - Elasticsearch v6 以降は、全文検索= text、完全一致/集計= keyword が基本。
#      - 本例の "content" は全文検索を想定し、type="text" としている。
#        ※ 完全一致や集計が必要なら multi-fields（例: "content.raw": keyword）を別途追加する設計が定石。
#   2) analyzer / search_analyzer
#      - index 時（投入時）: analyzer（ここでは "jpn-index"）でトークナイズ・正規化されて保存される。
#      - search 時（問い合わせ）: search_analyzer（"jpn-search"）でクエリ側のみ前処理（同義語展開など）を行う。
#        → 「インデックスはプレーン、検索で想起を伸ばす」設計により観測性と運用性を確保。
#   3) put_mapping の制約
#      - 既存フィールドの analyzer を後から変更することはできない（エラーになる/無視される）。
#        → 変更が必要な場合は「新インデックス作成 → reindex → エイリアス切替」が基本。
#      - 既存インデックスへの put_mapping は「新規フィールド追加」など限定的な変更のみ安全。
#   4) 依存関係
#      - "jpn-index" / "jpn-search" はインデックス settings.analysis に既に存在している必要がある
#        （前のステップで create の settings に定義済みであること）。
#   5) タイプレス・マッピング
#      - 7.x 以降は mapping type が廃止され、現行の書き方（type-less）で良い。
# -----------------------------------------------------------------------------------

mapping = {
    "properties": {
        "content": {
            # v6+ では全文検索対象は "text"、完全一致型は "keyword"
            # （v5 までの "string" は廃止済み）
            "type": "text",
            # インデックス生成時（文書投入時）のアナライザ
            # - "jpn-index": kuromoji + 各種フィルタ（同義語なし運用が一般的）
            "analyzer": "jpn-index",
            # 検索時（クエリ解析時）に用いるアナライザ
            # - "jpn-search": 同義語展開などで想起を高める検索専用チェーン
            "search_analyzer": "jpn-search",
            # 【発展（必要なら追加）】
            # "fields": {
            #   "raw": {
            #     "type": "keyword",
            #     "ignore_above": 256
            #     # 正規化が必要なら normalizer を定義して付与（例: lowercase）
            #   }
            # }
        }
    }
}

# マッピングの適用
# 注意:
# - 既に "content" が存在していて analyzer を変える等は不可。必要なら新インデックスを作成して reindex する。
# - Python クライアント 7/8 系では body=... での指定が一般的（8.x では query=... 等の新APIもあり）。
es.indices.put_mapping(index=jp_index, body=mapping)

# 【確認のヒント（必要時・参考）】
# cur = es.indices.get_mapping(index=jp_index)
# from pprint import pprint; pprint(cur[jp_index]["mappings"]["properties"].get("content"))

In [ ]:
# リスト 3.3.13 日本語文書の投入（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・前段で定義した日本語用インデックス（jp_index）に対して、サンプル文書を投入（indexing）する。
#   ・"content" フィールドには、マッピングで analyzer="jpn-index" / search_analyzer="jpn-search" が
#     紐付いている前提（リスト 3.3.12）。投入時に索引用アナライザでトークナイズ・正規化される。
#
# 理論メモ（設計の含意）:
#   ・全文検索の主対象は "content"。ここでは kuromoji＋ICU により、
#       - 文字正規化（全/半角・互換文字）
#       - 反復記号の展開（々/ゝ/ヽ）
#       - 原形化・品詞/ストップ語除去
#       - 数値正規化・長音正規化
#     を経て、**インデックス側の語彙**が安定化する。
#   ・検索時には "jpn-search" が適用され、（定義があれば）同義語展開で想起を伸ばす。
#   ・"title" や "name" は本スニペットではマッピング未定義のため **動的マッピング**に従う
#     （クラスター設定に依存して text / keyword サブフィールド等が自動付与される場合がある）。
#   ・`es.index(id=...)` は **同一IDへ再投入で上書き**（バージョン更新）となる。
#     既存IDがある場合にエラーとしたいなら `op_type="create"` を用いる。
#   ・直後に検索する検証では `refresh="wait_for"` を付けるか、投入後に `es.indices.refresh()` を呼ぶ。
# -----------------------------------------------------------------------------------

bodys = [
    {
        "title": "山田太郎の紹介",
        "name": {"last": "山田", "first": "太郎"},
        # 例: 「スシ」表記は icu_normalizer（互換/全半角）と kuromoji_stemmer 等で揺れ吸収
        #     同義語で「寿司, すし」を束ねたい場合は synonyms_filter に登録して検索側で展開するのが定石
        "content": "スシが好物です。犬も好きです。",
    },
    {
        "title": "田中次郎の紹介",
        "name": {"last": "田中", "first": "次郎"},
        # 例: 「だいすき/大好き」揺れは icu_normalizer と原形化+ストップ語除去である程度吸収
        "content": "そばがだいすきです。ねこも大好きです。",
    },
    {
        "title": "渡辺三郎の紹介",
        "name": {"last": "渡辺", "first": "三郎"},
        # 固有名詞（「はやぶさ」など）は NEologd やユーザー辞書を用意すると分割・品詞付与が安定
        "content": "天ぷらが好きです。新幹線はやぶさのファンです。",
    },
]

# 文書投入
# - enumerate の開始は 0。既に同IDが存在する環境では上書きになる点に注意（学習用としては妥当）。
# - ES 8.x の Python クライアントでは `document=` 引数が推奨（`body=` は後方互換）。
for i, body in enumerate(bodys):
    es.index(index=jp_index, id=i, body=body)
    # 例: 衝突回避（既存IDがあれば失敗させたい）場合
    # es.index(index=jp_index, id=i, document=body, op_type="create")

# 検証直後に検索する場合は refresh を明示（投入→即検索の不可視を防ぐ）
# es.indices.refresh(index=jp_index)

# （発展）
# - 大量投入時は helpers.bulk を利用して高速化し、明示 refresh（またはエイリアス切替）で可視化を制御する。
# - "title" や "name" にも検索要件があるなら、専用マッピング（text/keyword や search_as_you_type 等）を付与する。

In [ ]:
# リスト 3.3.14 日本語文書の検索（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・jp_index に投入済みの日本語文書から、全文検索（match）で「ｽｼ」を検索する。
#   ・検索クエリ側にはフィールドの search_analyzer（= "jpn-search"）が適用されるため、
#     文字正規化（icu_normalizer）→ 反復記号展開 → 形態素解析（kuromoji）→ 原形化/品詞・stop除去
#     → 数値/長音正規化 →（同義語：未定義なら no-op）という流れで前処理される。
#
# 理論メモ:
#   ・match は **全文検索**。`content` が text 型であればトークン化・正規化された語彙に対して BM25 で照合。
#   ・本例のクエリ文字列 "ｽｼ"（半角カナ）は、search_analyzer の **icu_normalizer** により
#     全角カタカナ「スシ」相当に正規化されるため、インデックス側（"jpn-index" で正規化済み）の
#     「スシ」等と一致しやすくなる（= 表記ゆれ吸収）。
#   ・「寿司/すし/スシ」を横断して想起を伸ばしたい場合は、synonyms_filter に
#     "寿司, すし, スシ" などを登録し、**検索側**（jpn-search）で展開する設計が定石。
#
# 実務上の注意:
#   ・直前に index したばかりの文書は、refresh 前は検索に出ないことがある。
#     検証では `es.index(..., refresh="wait_for")` や `es.indices.refresh(index=jp_index)` を利用。
#   ・返却件数は既定 size=10。厳密件数が必要なら search に `track_total_hits=True` を付与。
#   ・8.x クライアントでは `body=` より `query=` 引数が推奨（後方互換で body も動作）。
# -----------------------------------------------------------------------------------

# 検索条件の設定
# - "ｽｼ" は icu_normalizer で「スシ」に正規化され、kuromoji で形態素解析された上でマッチングされる。
query = {"query": {"match": {"content": "ｽｼ"}}}

# （参考：8.x 推奨形。必要ならこちらを利用）
# res = es.search(index=jp_index, query={"match": {"content": "ｽｼ"}}, size=10, track_total_hits=True)

# 検索実行
res = es.search(index=jp_index, body=query)

# 結果表示
# - ensure_ascii=False: 日本語などを \u エスケープせずに可読表示
# - 実務では res["hits"]["hits"] を走査して _id/_score/_source を整形してログに残すと追跡しやすい
import json

print(json.dumps(res, indent=2, ensure_ascii=False))

# （発展：_source 抽出の最小例）
# hits = [{"_id": h["_id"], "_score": h["_score"], "_source": h["_source"]} for h in res["hits"]["hits"]]
# print(json.dumps(hits, indent=2, ensure_ascii=False))

# （発展：同義語の利用例）
# - synonyms_filter に "寿司, すし, スシ" を登録し、インデックスは jpn-index（同義語なし）、
#   検索は jpn-search（同義語あり）とすることで、表記を跨いでヒットさせつつ観測性を保つ。